**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
!sudo apt-get update &&  sudo apt-get install ffmpeg

Ign:1 http://deb.debian.org/debian stretch InRelease
Hit:2 http://security.debian.org stretch/updates InRelease
Hit:3 http://deb.debian.org/debian stretch-updates InRelease                   
Hit:4 http://deb.debian.org/debian stretch-backports InRelease                 
Hit:5 http://deb.debian.org/debian stretch Release                             
Hit:7 http://packages.cloud.google.com/apt gcsfuse-stretch InRelease           
Hit:8 https://download.docker.com/linux/debian stretch InRelease
Hit:9 https://nvidia.github.io/libnvidia-container/debian9/amd64  InRelease
Hit:10 https://nvidia.github.io/nvidia-container-runtime/debian9/amd64  InRelease
Hit:11 http://packages.cloud.google.com/apt cloud-sdk-stretch InRelease
Hit:12 https://nvidia.github.io/nvidia-docker/debian9/amd64  InRelease
Hit:13 https://deb.nodesource.com/node_11.x stretch InRelease
Hit:6 https://packages.cloud.google.com/apt kubernetes-xenial InRelease
Hit:14 http://packages.cloud.google.com/apt google-compute-engine-st

In [15]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath("/opt/anaconda3/lib/python3.7/site-packagesffmpeg/")
import skvideo.io

import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [16]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [17]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function makes it possible to determine if the agent follows the principles that he has learned without improving or if he tries a random action that will potentially be useful to him or serve him.

Epsilon allows the agent to tempt you to improve it. A large epsilon will reflect an agent who is often trying to improve by trying a random action but will make his learning less stable. 
A small espilon represent an agent that is slow to learn by trying less random action but make his learning more stable.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [24]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        #skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        pass

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [71]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Board is the array that contains information about the bonuses (cheese cells) and maluses (poisonous cells) on the array. This matrix evolves according to where the agent walks.

Position is used to define the position of the agent. The agent is at the position where the value is 1. The values -1 at the borders allow to delimit the array by inciting the agent to avoid the borders.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [26]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass
    
    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [27]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {:04.1f}/{:04.1f} | Average score ({:05.2f})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [28]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 09.0/16.0 | Average score (-7.00)
Win/lose count 05.0/12.0 | Average score (-7.00)
Win/lose count 13.0/14.0 | Average score (-5.00)
Win/lose count 10.5/12.0 | Average score (-4.12)
Win/lose count 08.5/16.0 | Average score (-4.80)
Win/lose count 13.5/13.0 | Average score (-3.92)
Win/lose count 13.0/13.0 | Average score (-3.36)
Win/lose count 09.5/07.0 | Average score (-2.62)
Win/lose count 06.5/13.0 | Average score (-3.06)
Win/lose count 06.5/09.0 | Average score (-3.00)
Final score: -3.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
\begin{equation*}
\begin{aligned}
Q^{\pi}(s,a)
&=\mathbb{E}_{p^\pi}[G(0)|s_0 =s, a_0 = a] \\
&= \mathbb{E}_{p^\pi}[r(s_0, a_0) + \gamma \sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1})|s_0=s, a_0 = a] \\
&= \mathbb{E}_{p^\pi}[r(s_0, a_0) + \gamma G(1)|s_0 =s, a_0 = a] \\
&=  \mathbb{E}_{p^\pi}[r(s_0, a_0) + \gamma E(G(1)|s_1, a_1)|s_0 =s, a_0 = a] \\
&=\sum_r p(r|s,a)r + \gamma \sum_{a'}\sum_{s'} p(s', a'|s,a)Q^{\pi}(s',a')\\
&=\sum_r\sum_{s'}\sum_{a'} p( s',a',r|s,a)r + \gamma \sum_{a'}\sum_{s'}\sum_{r} p(s',a',r|s,a)Q^{\pi}(s',a')\\
&=\sum_r\sum_{s'}\sum_{a'}p(s',a',r|s,a)(r + \gamma Q^{\pi}(s',a'))\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
\end{aligned}
\end{equation*}



We can now compute the optimal Q function:
\begin{equation*}
\begin{aligned}
Q^{*}(s,a)
&=\max_{\pi} Q^{\pi}(s,a)\\
&=\max_{\pi}\mathbb{E}_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
&=\max_{a'}\mathbb{E}_{(s')\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')]\\
&\text{and with the linear property of } \mathbb{E} \\
&=\mathbb{E}_{(s')\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^{\pi^*}(s',a')]\\
\end{aligned}
\end{equation*}


The goal here is for the agent to try to maximize his profit. Thus, one will tend to tell him to take this target, which is none other than maximum profit, as his objective.
\begin{equation*}
\begin{aligned}
\mathcal{L}(\theta)
&=\mathbb{E}_{(s')\sim \pi^*(.|s,a)}([(Q(s,a,\theta) - \text{target}(s']^2)\\
&=\mathbb{E}_{(s')\sim \pi^*(.|s,a)}([r(s,a)+\gamma \max_{a'}Q^{\pi^*}(s',a') -Q(s,a,\theta)]^2)\\
&=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{aligned}
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [31]:
 class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) >= self.max_memory:
            del self.memory[0]
        self.memory.append(m)
             
    def random_access(self):
        random_index = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [32]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [108]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def predict(self, state):
        return self.model.predict(state)[0]
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            state, next_state, action, reward, game_over = self.memory.random_access()
            input_states[i] = state
            target_q[i] = self.predict(state.reshape(1, 5,5,self.n_state))
            
            # Q = max_a' Q(s', a')
            Q = np.max(self.predict(next_state.reshape(1, 5,5,self.n_state)))
            if game_over:
                target_q[i, action] = reward
            else:
                # reward + gamma * max_a' Q(s', a')
                target_q[i, action] = reward + self.discount * Q
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(BatchNormalization(input_shape=(5,5,self.n_state,)))
        model.add(Flatten())
        model.add(Dense(16, activation = 'relu'))
        model.add(Dense(4, activation = 'sigmoid'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.summary()
        self.model = model
        
        
        
from keras.layers.core import Dense, Flatten
from keras.engine.input_layer import Input
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

In [72]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.005, epsilon = 0.01, memory_size=2000, batch_size = 8, n_state=2)
train(agent, env, epochs_train, prefix='fc_train')
#HTML(display_videos('fc_train10.mp4'))

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_17 (Batc (None, 5, 5, 2)           8         
_________________________________________________________________
flatten_18 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 16)                816       
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 68        
Total params: 892
Trainable params: 888
Non-trainable params: 4
_________________________________________________________________
Epoch 000/100 | Loss 0.0122 | Win/lose count 0/1.0 (-1.0)
Epoch 001/100 | Loss 0.0518 | Win/lose count 0/0 (0)
Epoch 002/100 | Loss 0.0211 | Win/lose count 0/0 (0)
Epoch 003/100 | Loss 0.0369 | Win/lose count 0.5/1.0 (-0.5)
Epoch 004/100 | Loss 0.0143 |

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [109]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.01,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
                
        model = Sequential()
        model.add(BatchNormalization(input_shape=(5,5,self.n_state,)))
        model.add(Conv2D(2, 3, data_format = 'channels_last', activation = 'relu'))
        model.add(Conv2D(5, 3, data_format = 'channels_last', activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4, activation = 'sigmoid'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.summary()
        self.model = model

In [75]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.005, epsilon = 0.1, memory_size=2000, batch_size = 8)
train(agent,env,epochs_train,prefix='cnn_train')
#HTML(display_videos('cnn_train10.mp4'))

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_19 (Batc (None, 5, 5, 2)           8         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 3, 3, 2)           38        
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 1, 1, 5)           95        
_________________________________________________________________
flatten_20 (Flatten)         (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 4)                 24        
Total params: 165
Trainable params: 161
Non-trainable params: 4
_________________________________________________________________
Epoch 000/100 | Loss 0.0266 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/100 | Loss 0.0106 | Win/lose count 1.0/1.0 (0.0)
Ep

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [77]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.005, epsilon = 0.1, memory_size=2000, batch_size = 8)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.005, epsilon = 0.1, memory_size=2000, batch_size = 8)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_22 (Batc (None, 5, 5, 2)           8         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 3, 3, 2)           38        
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 1, 1, 5)           95        
_________________________________________________________________
flatten_23 (Flatten)         (None, 5)                 0         
_________________________________________________________________
dense_36 (Dense)             (None, 4)                 24        
Total params: 165
Trainable params: 161
Non-trainable params: 4
_________________________________________________________________
Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Out

In [115]:
env = Environment(grid_size=size, max_time=T,temperature=0.6)
agent_cnn = DQN_CNN(size, lr=.005, epsilon = 0.1, memory_size=2000, batch_size = 8)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.005, epsilon = 0.1, memory_size=2000, batch_size = 8)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_35 (Batc (None, 5, 5, 2)           8         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 3, 3, 2)           38        
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 1, 1, 5)           95        
_________________________________________________________________
flatten_36 (Flatten)         (None, 5)                 0         
_________________________________________________________________
dense_50 (Dense)             (None, 4)                 24        
Total params: 165
Trainable params: 161
Non-trainable params: 4
_________________________________________________________________
Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Out

Agents tend to stay on the cells they've already visited to make sure they don't stumble upon a poisonous cell. The network with CNNs is better than the one with fully connected layers. 

In [ ]:
#HTML(display_videos('cnn_test10.mp4'))

In [ ]:
#HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [137]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    best_previous_loss = 100
    epsilon_discount = 0.5
    min_epsilon = 0.001
    patience = 5
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        # set epsilon only if the loss get worth
        if loss < best_previous_loss:
            best_previous_loss = loss
            patience_counter = 0
        else:
            patience_counter += 1 
            if patience_counter > patience and min_epsilon < agent.epsilon:
                agent.set_epsilon(max(epsilon_discount * agent.epsilon, min_epsilon))
                print('Set a new epsilon: {}'.format(agent.epsilon))
                
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:04.1f}/{:04.1f} ({:04.1f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [138]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        #skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        pass

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        # we add a new feature : the malus-position
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.t = self.t + 1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [139]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.005, epsilon = 0.1, memory_size=2000, batch_size = 8,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
#HTML(display_videos('cnn_train_explore10.mp4'))

Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_44 (Batc (None, 5, 5, 3)           12        
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 3, 3, 2)           56        
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 1, 1, 5)           95        
_________________________________________________________________
flatten_45 (Flatten)         (None, 5)                 0         
_________________________________________________________________
dense_60 (Dense)             (None, 4)                 24        
Total params: 187
Trainable params: 181
Non-trainable params: 6
_________________________________________________________________
Epoch 000/100 | Loss 0.0134 | Win/lose count 01.0/19.2 (-18.2)
Epoch 001/100 | Loss 0.0053 | Win/lose count 00.0/22.1 (-2

FileNotFoundError: [Errno 2] No such file or directory: 'cnn_train_explore10.mp4'

In [140]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 00.0/01.0 | Average score (-1.00)
Win/lose count 00.5/00.0 | Average score (-0.25)
Win/lose count 00.0/01.0 | Average score (-0.50)
Win/lose count 00.0/00.0 | Average score (-0.38)
Win/lose count 00.0/00.0 | Average score (-0.30)
Win/lose count 00.0/00.0 | Average score (-0.25)
Win/lose count 00.0/00.0 | Average score (-0.21)
Win/lose count 00.0/00.0 | Average score (-0.19)
Win/lose count 00.0/00.0 | Average score (-0.17)
Win/lose count 00.0/00.0 | Average score (-0.15)
Win/lose count 00.0/00.0 | Average score (-0.14)
Win/lose count 00.0/00.0 | Average score (-0.12)
Win/lose count 00.0/00.0 | Average score (-0.12)
Win/lose count 00.0/00.0 | Average score (-0.11)
Win/lose count 00.0/00.0 | Average score (-0.10)
Win/lose count 00.5/00.0 | Average score (-0.06)
Win/lose count 00.0/00.0 | Average score (-0.06)
Win/lose count 00.0/00.0 | Average score (-0.06)
Win/lose count 00.0/00.0 | Average score (-0.05)
Win/lose count 00.0/00.0 | Average score (-0.05)
Final score: -0.05


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***